# 5.2 Korpusmethoden in scikit-learn

- CountVectorizer, TF-IDF-Vectorizer
- Train-Test-Splits von Korpora


- scikit-learn User Guide: https://scikit-learn.org/stable/user_guide.html



## 1. scikit-learn 

- die Python-Library scikit-learn bietet eine Vielzahl von Machine Learning-Methoden und Algorithmen an (*supervised* und *unsupervised*)
- stellt auch Korpus-Datensätze zum Trainieren und Testen von NLP-Klassifikatoren zur Verfügung (z.B. '20 Newsgroup text dataset')
- Vielzahl von Hilfsmethoden, etwa für: 
  - Umwandlung von Korpora in eine Matrix von Counts (Vectorizer) zur Feature Extraction
  - Aufteilen von Korpusdatensätzen in Test- und Trainingssets


In [1]:
import sklearn
import pandas as pd
import numpy as np

---
## 2. CountVectorizer()

- extrahiert eine Dokument-Term-Matrix mit Frequenzen des Vokabulars (Types der Wörter im Korpus) für jedes Dokument, indem die Textdokumente tokenisiert und die Frequenzen der Worttypes berechnet werden (*term frequency*)
- dient im Kontext von scikit-learn primär der Extraktion von Features für die Textklassifikation (siehe unten, Anwendungsbeispiel)
- kann auch zur Berechnung von Frequenzlisten verwendet werden


In [2]:
corpus = [
'This is the first document.',
'This document is the second document.',
'And this is the third one.',
'Is this the first document?',
'This is another document',
]

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(corpus) #X = vectors
print(X.shape)

(5, 10)


In [4]:
X

<5x10 sparse matrix of type '<class 'numpy.int64'>'
	with 25 stored elements in Compressed Sparse Row format>

In [5]:
X.todense()

matrix([[0, 0, 1, 1, 1, 0, 0, 1, 0, 1],
        [0, 0, 2, 0, 1, 0, 1, 1, 0, 1],
        [1, 0, 0, 0, 1, 1, 0, 1, 1, 1],
        [0, 0, 1, 1, 1, 0, 0, 1, 0, 1],
        [0, 1, 1, 0, 1, 0, 0, 0, 0, 1]])

In [6]:
df = pd.DataFrame(X.toarray(), 
                  index=["Document_1","Document_2","Document_3","Document_4","Document_5"],
                  columns=vectorizer.get_feature_names_out())
df

,and,another,document,first,is,one,second,the,third,this
Document_1,0,0,1,1,1,0,0,1,0,1
Document_2,0,0,2,0,1,0,1,1,0,1
Document_3,1,0,0,0,1,1,0,1,1,1
Document_4,0,0,1,1,1,0,0,1,0,1
Document_5,0,1,1,0,1,0,0,0,0,1


---
## 3. Train-Test-Splits 


- für Training (`train`), Evaluierung (`test`) und ggf. Fehleranalyse (`dev`) von Klassifikatoren müssen die Korpusdaten gemischt (*shuffle*) und in Teilmengen aufgeteilt werden
- scikit-learn stellt hierfür Methoden zur Verfügung

<img src="https://www.nltk.org/images/corpus-org.png" width=33% /> 

https://www.nltk.org/book/ch06.html#fig-corpus-org 


### 3.1 Train-Test-Split mit scikit-learn

### train_test_split()

In [7]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)
print(len(train),len(test))

4 1


In [8]:
test

,and,another,document,first,is,one,second,the,third,this
Document_2,0,0,2,0,1,0,1,1,0,1


### 3. 2 Train-Test-Split mit pandas

Auch mit pandas können solche Korpus-Splits durchgeführt werden:


### sample()

In [9]:
train = df.sample(frac=0.8,random_state=42)
test = df.drop(train.index)
print(len(train),len(test))

4 1


In [10]:
test

,and,another,document,first,is,one,second,the,third,this
Document_4,0,0,1,1,1,0,0,1,0,1


### 3.3 Train-Test-Dev-Split mit numpy und pandas

In [11]:
# 60%, 20%, 20% split
train, validate, test = np.split(df.sample(frac=1,random_state=42), [int(.6*len(df)), int(.8*len(df))]) 
print(len(train),len(validate),len(test))

3 1 1


---
---
## *4. Anwendungsbeispiel: Textklassifikation mit TF-IDF-Feature-Extraction*

*nicht klausurrelevant!*


### Feature Extraction für Textklassifikation

<img src="https://www.nltk.org/images/supervised-classification.png" width=38%>

https://www.nltk.org/book/ch06.html#fig-supervised-classification
>Supervised Classification. (a) During training, a ***feature extractor*** is used to convert each input value to a feature set. These feature sets  \[...\] capture the basic information about each input that should be used to classify it \[...\]. Pairs of feature sets and labels are fed into the machine learning algorithm to generate a model. (b) During prediction, the same feature extractor is used to convert unseen inputs to feature sets. These feature sets are then fed into the model, which generates predicted labels.

### Anwendungsbeispiel

- Im Folgenden wird das 20newsgroups-Korpus geladen und zum **Training eines tfidf-Klassifikators für Textsorten** verwendet:
  - das 20newsgroups ist Teil der Ressourcen von scikit-learn 
  - es ist bereits in Train-Test-Subsets aufgeteilt und nach Textsorten filterbar
  - im Beispiel: Einschränkung auf 4 Genres/Textsorten
  
https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html


In [12]:
from sklearn.datasets import fetch_20newsgroups

In [13]:
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [14]:
len(twenty_train.data)
len(twenty_train.filenames)

2257

In [15]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton


###  Feature Extraction (TF-IDF-Keywords)

Mit Hilfe des `TfidfVectorizer` werden die Textdokumente tokenisiert und für die Worttypes die entsprechende TF-IDF-Features als für die Dokumente relevanten Keywords berechnet, indem die Termfrequenzen (TF) mit der inversen Häufigkeit in allen Dokumenten (IDF) gewichtet werden.

Ergebnis ist wieder eine Dokument-Term-Matrix (wie oben beim `CountVectorizer`):

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(twenty_train.data)
X_train_tfidf.shape

(2257, 35788)

https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html#converting-text-to-vectors:

> The extracted TF-IDF vectors are very sparse: 
> an average of 162 non-zero components by sample in a more than 35000-dimensional space (less than .5% non-zero features; 162/35788) 



In [17]:
#vectors.nnz / float(vectors.shape[0])
X_train_tfidf.nnz / float(X_train_tfidf.shape[0])

162.1116526362428

In [18]:
#Get the count of explicitly-stored values (nonzeros)
X_train_tfidf.nnz

365886

### Training eines Naive-Bayes-Klassifikators und Prediction

In [19]:
#import classifier
from sklearn.naive_bayes import MultinomialNB

In [20]:
#Training a classifier
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [21]:
#predict
docs_new = ['God is love', 'OpenGL on the GPU is fast']

X_new_tfidf = vectorizer.transform(docs_new)
predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


### Erstellen einer Pipeline und Evaluierung

In [22]:
#Building a pipeline with MultinomialNB-classifier:
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
    ])

In [23]:
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [24]:
#Evaluation of the performance on the test set
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.8348868175765646